In [110]:
import json
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

stemmer=PorterStemmer()

def Tokenize(sentence):
    return nltk.word_tokenize(sentence)

def Stem(word):
    """Stems a word using the Porter Stemmer."""
    return stemmer.stem(word.lower())

def Bag_Of_Word(tokenize_sentence,all_words):
    tokenize_sentence=[Stem(w) for w in tokenize_sentence]
    bag=np.zeros(len(all_words),dtype=np.float32)

    for ind,w in enumerate(all_words):
        if w in tokenize_sentence:
            bag[ind]=1.0
    return bag



with open('G:/chatbot/intents.json','r') as f:
    intents=json.load(f)

all_words=[]
tags=[]
xy=[]

for intent in intents['intents']:
    tag=intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w=Tokenize(pattern)
        all_words.extend(w)
        xy.append((w,tag))

# Preprocess the data
ignore_words = ['?', '!', '.', ',']  # List of words to ignore
all_words = [Stem(w) for w in all_words if w not in ignore_words]  # Stem and remove ignored words
all_words = sorted(set(all_words))  # Remove duplicates and sort alphabetically
tags = sorted(set(tags))  # Remove duplicates and sort alphabetically


x_train=[]
y_train=[]

for (pattern_sentence, tag) in xy:
    bag = Bag_Of_Word(pattern_sentence, all_words)
    x_train.append(bag)

    label = tags.index(tag)
    y_train.append(label)


# Convert lists to numpy arrays

x_train = np.array(x_train)
y_train = np.array(y_train)

In [111]:
print(f"number of samples (patterns) and the number of features (words in the bag-of-words representation)  = {x_train.shape}")

print(f"number of labels corresponding to your patterns in x_train {y_train.shape}")

number of samples (patterns) and the number of features (words in the bag-of-words representation)  = (26, 54)
number of labels corresponding to your patterns in x_train (26,)


### Hyperparameters for training the neural network

In [112]:
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(x_train[0])
hidden_size = 8
output_size = len(tags)
print(f"input size = {input_size} and output_size = {output_size}")

input size = 54 and output_size = 7


In [113]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

In [114]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        # Forward pass through the network with ReLU activation
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        # use cross-entrophy function
        return out

In [115]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [116]:
model=NeuralNet(input_size,hidden_size,output_size).to(device)

### Loss and optimizer

In [117]:
# Define loss function (cross-entropy for multi-class classification)
criterion=nn.CrossEntropyLoss()

# Define optimizer (Adam with specified learning rate)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

### training loop

In [118]:
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        # Backward and optimizer step
        optimizer.zero_grad()      # Clear gradients before backpropagation
        loss.backward()     # Backpropagate to calculate parameter gradients
        optimizer.step()    # Update network weights based on gradients and optimizer

    # Print loss every 100 epochs
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Print final loss after training
print(f'final loss: {loss.item():.4f}')


Epoch [100/1000], Loss: 0.8033
Epoch [200/1000], Loss: 0.0607
Epoch [300/1000], Loss: 0.0142
Epoch [400/1000], Loss: 0.0065
Epoch [500/1000], Loss: 0.0027
Epoch [600/1000], Loss: 0.0022
Epoch [700/1000], Loss: 0.0003
Epoch [800/1000], Loss: 0.0009
Epoch [900/1000], Loss: 0.0017
Epoch [1000/1000], Loss: 0.0002
final loss: 0.0002


## Save model

In [119]:
# Save the trained model's state and other relevant information

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


## use saved model

In [ ]:
import random
import json
import torch


def Tokenize(sentence):
    return nltk.word_tokenize(sentence)
def Bag_Of_Word(tokenize_sentence,all_words):
    tokenize_sentence=[Stem(w) for w in tokenize_sentence]
    bag=np.zeros(len(all_words),dtype=np.float32)

    for ind,w in enumerate(all_words):
        if w in tokenize_sentence:
            bag[ind]=1.0
    return bag



class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        # use cross-entrophy function
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('G:/chatbot/intents.json','r') as f:
    intents=json.load(f)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "MyBot"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = Tokenize(sentence)
    X = Bag_Of_Word(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")


Let's chat! (type 'quit' to exit)


You:  hi


MyBot: Hello, thanks for visiting
